In [9]:
import pandas as pd
import lxml
import glob
from lxml import etree
from bs4 import BeautifulSoup
import re 
import requests
from urllib.parse import quote
import sys


ns = "{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}"
cards = './urdu_cards_pages/1032971/U2_for_Transkribus/page/*.xml'

In [10]:
cards_samples = []
for card in glob.glob(cards):
    cards_samples.append(card)

cards_samples

['./urdu_cards_pages/1032971/U2_for_Transkribus/page/0024_24567998184.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0036_24567999964.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0040_24568000444.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0034_24567999574.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0037_24568000074.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0018_24567973754.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0021_24567997914.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0022_24567997984.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0044_24568001044.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0015_24567973364.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0013_24567973014.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0028_24567998604.xml',
 './urdu_cards_pages/1032971/U2_for_Transkribus/page/0023_24567998094.xml',
 './urdu_car

In [11]:
metadata = pd.DataFrame()
shelfmarks = []
authors = []
titles = []
query = []
query_results = []

for card in cards_samples:
    with open(card, "r") as file:
        print(card)
        
        soup = BeautifulSoup(file, 'xml')
        shelfmark = soup.select('TextRegion[custom*=shelfmark]')[0].find_all('TextLine', recursive=False)[0].find_all('TextEquiv', recursive=False)[0].find_all('Unicode', recursive=False)[0]
        shelfmark = shelfmark.get_text()
        author = soup.select('TextRegion[custom*=author]')[0].find_all('TextLine', recursive=False)[0].find_all('TextEquiv', recursive=False)[0].find_all('Unicode', recursive=False)[0]
        author = author.get_text()
        title = soup.select('TextRegion[custom*=title]')[0].find_all('TextLine', recursive=False)[0].find_all('TextEquiv', recursive=False)[0].find_all('Unicode', recursive=False)[0]
        title = title.get_text()
        
        #SAVE METADATA
        shelfmarks.append(shelfmark)
        authors.append(author)
        titles.append(title)
        
        surname = author.split(',')[0]
        surname_encoded = quote(surname)
        
        #sys.stdout = open(card.split('/')[-1]+'.txt','wt')

        #INITIAL QUERY
        url = 'https://www.worldcat.org/search?q=au%3A'+surname_encoded+'&qt=advanced&dblist=638'
        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        #CHECK NUMBER OF RESULTS
        try:
            results_info = soup.select('div[class=resultsinfo]')[0].find_all('td')[0].find_all('strong')[1].get_text()
            n_pages = int(float(results_info.replace(',', '')))//10
        except:
              n_pages = "NO RESULTS"
        
        query.append(surname)
        query_results.append(int(float(results_info.replace(',', ''))))
        
        
        #CONSTRUCT URLS
        '''urls = [url]  
        if n_pages > 1:
            if n_pages > 10:
                end = 11
            else:
                end=n_pages+1
            for i in range(1,end):
                urls.append(url+'&start='+str(i*10+1)+'&qt=page_number_link')
        
        print('===========')
        print("CARD TRANSCRIPTION")
        print('Shelfmark = '+ shelfmark)
        print('Author = '+ author)
        print('Title = '+ title)
        print()
        print("**PARAMETERS USED ON WORLDCAT**")
        print('Surname: ' + surname)
        print('N. results: ' + str(results_info))
        print('===========')
        print()
        print("RESULTS (limited to max 100):")

        n = 1

        for u in urls:
            page = requests.get(u)
            soup = BeautifulSoup(page.content, 'html.parser')
            results = soup.select('a[id*=result]')
            for result in results:
                s = str(result['href'])
                print(n)
                title = result.find('strong').text  
                print('Title: '+ title)
                ocloc_id = re.search(r'[0-9]{6,}', s)
                print('ocloc_id: '+ ocloc_id.group(0))
                url = 'http://www.worldcat.org/oclc/'+ocloc_id.group(0)
                print('URL: '+ url)
                print('===========')
                print()
                n +=1'''
                
                




./urdu_cards_pages/1032971/U2_for_Transkribus/page/0024_24567998184.xml
https://www.worldcat.org/search?q=au%3A%27ABD%20al-RAHMAN&qt=advanced&dblist=638
./urdu_cards_pages/1032971/U2_for_Transkribus/page/0036_24567999964.xml
https://www.worldcat.org/search?q=au%3A%27ABD%20al-SHAKUR&qt=advanced&dblist=638
./urdu_cards_pages/1032971/U2_for_Transkribus/page/0040_24568000444.xml
https://www.worldcat.org/search?q=au%3ANIYKZ%20%28Mubammad%20%27Abdullah%29&qt=advanced&dblist=638
./urdu_cards_pages/1032971/U2_for_Transkribus/page/0034_24567999574.xml
https://www.worldcat.org/search?q=au%3ANASIM&qt=advanced&dblist=638
./urdu_cards_pages/1032971/U2_for_Transkribus/page/0037_24568000074.xml
https://www.worldcat.org/search?q=au%3A%27ABD%20al-WAHID&qt=advanced&dblist=638
./urdu_cards_pages/1032971/U2_for_Transkribus/page/0018_24567973754.xml
https://www.worldcat.org/search?q=au%3AISHRAT&qt=advanced&dblist=638
./urdu_cards_pages/1032971/U2_for_Transkribus/page/0021_24567997914.xml
https://www.worldc

In [12]:
metadata['shelfmark'] = shelfmarks
metadata['author'] = authors
metadata['title'] = titles
metadata['query'] = query
metadata['query_results'] = query_results
metadata.to_csv('metadata.csv', index=False)